# Odległość edycyjna

In [5]:
# miacierz odlgłości edycyjnej - tablica w której dynamicznie zapisujemy 
# odległość edycyjną między prefiksami słów "s" i "t"
def levenshtein_distance_matrix(s, t):
    m, n = len(s), len(t)
    # inicjalizacja macierzy
    d = [[0] * (n + 1) for i in range(m + 1)]
    for i in range(m + 1):
        d[i][0] = i
    for j in range(n + 1):
        d[0][j] = j
    # wypełnienie macierzy
    for j in range(1, n + 1):
        for i in range(1, m + 1):
            if s[i - 1] == t[j - 1]:
                d[i][j] = d[i - 1][j - 1]
            else:
                d[i][j] = min(d[i - 1][j], d[i][j - 1], d[i - 1][j - 1]) + 1
    return d

# odległość edycyjna między "s" i "t"
def levenshtein_distance(s, t):
    m, n = len(s), len(t)
    return levenshtein_distance_matrix(s,t)[m][n]



## Wizualizacja

In [52]:
from enum import Enum

# 4 typy rodzaju edycji tekstu (z czego jeden trywialny: NONE)
class EditType(Enum):
    NONE = 0
    INSERT = 1
    DELETE = 2
    REPLACE = 3

# wszystkie zmiany prowadzące ze słowa "s" na "t"
def transwer_word_edits(s,t):
    d = levenshtein_distance_matrix(s,t)
    i, j = len(s), len(t) 
    edits = []
    while i > 0 or j > 0:
        if i > 0 and j > 0 and d[i - 1][j - 1] == d[i][j] and s[i - 1] == t[j - 1]:
            i, j = i - 1, j - 1
            edits.append((EditType.NONE, s[i]))
        elif i > 0 and d[i - 1][j] == d[i][j] - 1:
            i -= 1
            edits.append((EditType.DELETE, s[i]))
        elif j > 0 and d[i][j - 1] == d[i][j] - 1:
            j -= 1
            edits.append((EditType.INSERT, t[j]))
        elif i > 0 and j > 0 and d[i - 1][j - 1] == d[i][j] - 1:
            i, j = i - 1, j - 1
            edits.append((EditType.REPLACE, (s[i], t[j])))
    return edits[::-1]

# wizualizacja od lewej do prawej 
def transwer_word_wizualation(s, t):
    edits = transwer_word_edits(s,t)
    curr_s = s[:]+" "
    i=0
    for edType, edLett in edits:
        if edType == EditType.NONE:
            pass            
        elif edType == EditType.DELETE:
            print(f"{s} --(Delete letter '{edLett}' at index {i})--> {s[:i] + s[i+1:]}")
            s = s[:i] + s[i+1:]
            i -= 1
        elif edType == EditType.INSERT:
            print(f"{s} --(Inserted letter '{edLett}' at index {i})--> {s[:i] + edLett + s[i:]}")
            s = s[:i] + edLett + s[i:]
        elif edType == EditType.REPLACE:
            print(f"{s} --('{edLett[0]}' Replace with '{edLett[1]}' at index {i})--> {s[:i] + edLett[1] + s[i+1:]}")
            s = s[:i] + edLett[1] + s[i+1:]
        i += 1


przykładowe wywołania

In [53]:
s1,s2 = "los", "kloc"
print(f"{s1} --> {s2} in {levenshtein_distance(s1, s2)} steps")
transwer_word_wizualation(s1, s2)
print()

s1,s2 = "Łódź", "Lodz"
print(f"{s1} --> {s2} in {levenshtein_distance(s1, s2)} steps")
transwer_word_wizualation(s1, s2)
print()

s1,s2 = "kwintesencja", "quintessence"
print(f"{s1} --> {s2} in {levenshtein_distance(s1, s2)} steps")
transwer_word_wizualation(s1, s2)
print()

s1,s2 = "ATGAATCTTACCGCCTCG", "ATGAGGCTCTGGCCCCTG"
print(f"{s1} --> {s2} in {levenshtein_distance(s1, s2)} steps")
transwer_word_wizualation(s1, s2)
print()


los --> kloc in 2 steps
los --(Inserted letter 'k' at index 0)--> klos
klos --('s' Replace with 'c' at index 3)--> kloc

Łódź --> Lodz in 3 steps
Łódź --('Ł' Replace with 'L' at index 0)--> Lódź
Lódź --('ó' Replace with 'o' at index 1)--> Lodź
Lodź --('ź' Replace with 'z' at index 3)--> Lodz

kwintesencja --> quintessence in 5 steps
kwintesencja --('k' Replace with 'q' at index 0)--> qwintesencja
qwintesencja --('w' Replace with 'u' at index 1)--> quintesencja
quintesencja --(Inserted letter 's' at index 6)--> quintessencja
quintessencja --('j' Replace with 'e' at index 11)--> quintessencea
quintessencea --(Delete letter 'a' at index 12)--> quintessence

ATGAATCTTACCGCCTCG --> ATGAGGCTCTGGCCCCTG in 7 steps
ATGAATCTTACCGCCTCG --('A' Replace with 'G' at index 4)--> ATGAGTCTTACCGCCTCG
ATGAGTCTTACCGCCTCG --('T' Replace with 'G' at index 5)--> ATGAGGCTTACCGCCTCG
ATGAGGCTTACCGCCTCG --(Inserted letter 'C' at index 8)--> ATGAGGCTCTACCGCCTCG
ATGAGGCTCTACCGCCTCG --('A' Replace with 'G' at index 